In [2]:
!pip install dostoevsky
!python3 -m dostoevsky download fasttext-social-network-model

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 1.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 8.4 MB/s eta 0:00:00:00:0100:01
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=d4efb0a4349f99436516cfe52538a523bad6b6b9e2161a136005393773f682d2
  Stored in directory: /config/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [32]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

messages = [
    'привет',
    'я люблю тебя!!',
    'This is a very important thing!'
]

results = model.predict(messages, k=2)

formatted_results = []
for message, sentiment in zip(messages, results):
    formatted_result = {
        'positive': sentiment.get('positive', -2),
        'neutral': sentiment.get('neutral', -2),
        'negative': sentiment.get('negative', -2),
        'speech': sentiment.get('speech', -2),
        'skip': sentiment.get('skip', -2),
    }
    formatted_results.append({message: formatted_result})

In [33]:
formatted_results

[{'привет': {'positive': -2,
   'neutral': -2,
   'negative': -2,
   'speech': 1.0000100135803223,
   'skip': 0.0020607432816177607}},
 {'я люблю тебя!!': {'positive': 0.9886782765388489,
   'neutral': -2,
   'negative': -2,
   'speech': -2,
   'skip': 0.005394937004894018}},
 {'This is a very important thing!': {'positive': 0.3276783227920532,
   'neutral': 0.7606606483459473,
   'negative': -2,
   'speech': -2,
   'skip': -2}}]

In [13]:
def analyze_sentiment(messages):
    tokenizer = RegexTokenizer()
    model = FastTextSocialNetworkModel(tokenizer=tokenizer)
    
    results = model.predict(messages, k=2)

    formatted_results = []
    for message, sentiment in zip(messages, results):
        formatted_result = {
            'positive': sentiment.get('positive', -2),
            'neutral': sentiment.get('neutral', -2),
            'negative': sentiment.get('negative', -2),
            'speech': sentiment.get('speech', -2),
            'skip': sentiment.get('skip', -2),
        }
        formatted_results.append({message: formatted_result})

    return formatted_results

In [15]:
results = analyze_sentiment(messages)
results

[{'привет': {'positive': -2,
   'neutral': -2,
   'negative': -2,
   'speech': 1.0000100135803223,
   'skip': 0.0020607432816177607}},
 {'я люблю тебя!!': {'positive': 0.9886782765388489,
   'neutral': -2,
   'negative': -2,
   'speech': -2,
   'skip': 0.005394937004894018}},
 {'This is a very important thing!': {'positive': 0.3276783227920532,
   'neutral': 0.7606606483459473,
   'negative': -2,
   'speech': -2,
   'skip': -2}}]

In [15]:
import sys
import json
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

def person_catcher(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    #создадим список в который запишем всех людей, которые попали в текст
    persons = []

    for span in doc.spans:
        if span.type == PER:
            persons.append(span.normal)
    # Учесть, что может не быть людей в сообщении
    # Доделать красивый вывод сообщения
    return persons


In [16]:
person_catcher('Владимир Путин едет на Сахалин')

[None]

: 

In [ ]:

if name == "main":
    text = sys.argv[1:]
    results = person_catcher(text)
    print(json.dumps(results))

In [15]:
from langdetect import detect

text = "Привет, проверяем"
language = detect(text)
print(language)

ru


In [13]:
pip install language_data

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import sys
import json

import torch
from transformers import BertForSequenceClassification, AutoTokenizer

LABELS = ['Нейтральный', 'Позитивный', 'Негативный', 'Восторг', 'Страх', 'Злость', 'Отвращение']
tokenizer = AutoTokenizer.from_pretrained('Aniemore/rubert-tiny2-russian-emotion-detection')
model = BertForSequenceClassification.from_pretrained('Aniemore/rubert-tiny2-russian-emotion-detection')

def predict_emotion(text: str) -> str:
    """
        We take the input text, tokenize it, pass it through the model, and then return the predicted label
        :param text: The text to be classified
        :type text: str
        :return: The predicted emotion
    """
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
        
    return LABELS[predicted[0]]


In [11]:
predict_emotion('Пипец ты мудачелло, сук, люблю Молодец!!')


'Позитивный'

In [25]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 9.1 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 KB 11.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 KB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0
Note: you may need to restart the kernel to use updated packages.
